# 02 : GPU Check

This is a simple test to see if the GPU is available and working correctly.

- https://stackoverflow.com/questions/76581229/is-it-possible-to-check-if-gpu-is-available-without-using-deep-learning-packages
- https://docs.mlrun.org/en/v1.7.2/runtimes/configuring-job-resources.html
- https://docs.k3s.io/advanced#nvidia-container-runtime

In [1]:
import mlrun

> 2025-07-28 17:42:52,073 [info] Server and client versions are not the same but compatible: {"parsed_client_version":"Version(major=1, minor=7, patch=2, prerelease=None, build=None)","parsed_server_version":"Version(major=1, minor=9, patch=1, prerelease=None, build=None)"}


In [2]:
# Show the API server URL
mlrun.get_run_db()

HTTPRunDB('http://dragon.local:30070')

In [3]:
# Set the base project name
project_name = "mlrun-demo"

# Initialize the MLRun project object
project = mlrun.get_or_create_project(
    name=project_name, 
    context="./",
    user_project=True)

# Display the current project name
project_name = project.metadata.name
print(f'Full project name: {project_name}')

> 2025-07-28 17:42:52,126 [info] Project loaded successfully: {"project_name":"mlrun-demo-johannes"}
Full project name: mlrun-demo-johannes


## Get GPU Function

In [4]:
%%writefile 02_get_gpu_info.py

import GPUtil
import subprocess

def get_gpu_info(context):    
    gpus = GPUtil.getGPUs()
    gpu_info = []
    for gpu in gpus:
        gpu_info.append({
            'id': gpu.id,
            'name': gpu.name,
            'load': gpu.load,
            'memory_total': gpu.memoryTotal,
            'memory_free': gpu.memoryFree,
            'memory_used': gpu.memoryUsed,
        })

    print(f"GPU Info v4: {gpu_info}")
    context.logger.info(f"GPU Info: {gpu_info}")

    # execute the nvidia-smi command on the cli to get detailed GPU info
    try:
        nvidia_smi_output = subprocess.check_output(['nvidia-smi'], universal_newlines=True)
        print("NVIDIA-SMI Output:")
        print(nvidia_smi_output)
        context.logger.info(f"NVIDIA-SMI Output:\n{nvidia_smi_output}")
    except Exception as e:
        error_msg = f"Error running nvidia-smi: {str(e)}"
        print(error_msg)
        context.logger.warning(error_msg)
    
    return gpu_info

Overwriting 02_get_gpu_info.py


## ML Run Function

In [ ]:
fn_gpu_check = project.set_function(
    func="02_get_gpu_info.py",
    name="gpu-check",
    tag="latest",
    kind="job",
    image="dragon:6500/mlrun/mlrun-gpu:1.7.2",
    handler="get_gpu_info",
    requirements=["GPUtil==1.4.0"])

# Then set the GPU resources on the function's spec
fn_gpu_check.with_limits(mem="2G", cpu=2, gpus=1)  # upper bound

# build the function
project.build_function(function='gpu-check')

> 2025-07-28 17:44:51,930 [warning] it is recommended to use k8s secret (specify secret_name), specifying the aws_access_key/aws_secret_key directly is unsafe


The `overwrite_build_params` parameter default will change from 'False' to 'True' in 1.8.0.


> 2025-07-28 17:44:52,206 [info] Started building image: .mlrun/func-mlrun-demo-johannes-gpu-check:latest
error checking push permissions -- make sure you entered the correct tag name, and that you are authenticated correctly, and try again: checking push permission for "dragon:6500/mlrun/func-mlrun-demo-johannes-gpu-check:latest": creating push check transport for dragon:6500 failed: Get "https://dragon:6500/v2/": tls: failed to verify certificate: x509: certificate signed by unknown authority


MLRunRuntimeError: Deploy failed

In [ ]:
# run the function locally
fn_gpu_check.run(
    local=False,
    handler="get_gpu_info",
    auto_build=True
)

In [ ]:
# Refresh the function from the saved .py file (this is instant)
project.set_function(func="02_get_gpu_info.py", name="gpu-check")

# run the function again [you can modify the py file and set the function again = this will not rebuild the entire thing] 
project.run_function(
    function='gpu-check',
    handler='get_gpu_info'
)